In [1]:
import os
import time
import pandas as pd
import numpy as np

from astropy.io import ascii
from astropy.coordinates import SkyCoord
from SPLASH.pipeline import Splash_Pipeline

# Load in data

In [2]:
# Load in 100 bts host grizy observations
try:
    bts_hosts = ascii.read('demo_bts_hosts.ecsv')
except FileNotFoundError:
    os.chdir('SPLASH_demo')
    bts_hosts = ascii.read('demo_bts_hosts.ecsv')
bts_hosts = bts_hosts[1:]  # drop annoying row
bts_df = bts_hosts.to_pandas()
bts_df.head()

,objID_3pi,raStack_3pi,decStack_3pi,primaryDetection_3pi,gKronMag_3pi,rKronMag_3pi,iKronMag_3pi,zKronMag_3pi,yKronMag_3pi,gKronMagErr_3pi,rKronMagErr_3pi,iKronMagErr_3pi,zKronMagErr_3pi,yKronMagErr_3pi,ps_score_3pi,SN_ra,SN_dec,sn_class,sn_redshift,ZTFID
0,148231465149294617,146.514940,33.528361,1,18.2387,17.9746,17.8750,17.8532,17.7821,0.004157,0.004871,0.004281,0.014407,0.019814,0.021125,146.514792,33.528250,SN II,0.03800,ZTF18aacemcn
1,153731349546771375,134.954717,38.108970,1,18.7478,19.0585,18.6466,18.8301,18.6221,0.006392,0.009475,0.004919,0.017235,0.038667,0.017411,134.954667,38.109056,SN II,0.07247,ZTF18aacnlxz
2,169101206490177628,120.649031,50.922461,1,16.5325,15.6522,15.2551,15.0064,14.7582,0.002335,0.002128,0.000867,0.001623,0.003512,0.075905,120.648958,50.922528,SN Ia,0.05295,ZTF18aadlaxo
3,159661026075384557,102.607536,43.053266,1,NaN,14.2309,13.1660,12.9816,12.8047,NaN,0.000836,0.000408,0.001003,0.001687,0.154649,102.607542,43.053222,SN IIn,0.01885,ZTF18aadmssd
4,146001295340652041,129.534082,31.667894,1,17.2694,16.9697,16.8672,16.8232,16.7636,0.004272,0.003413,0.005182,0.005542,0.014564,0.004667,129.533958,31.667917,SN II,0.03057,ZTF18aadsuxd


In [3]:
def get_angular_separation(ra1, dec1, ra2, dec2, unit1='deg', unit2='deg'):
    """Function to calculate angular separation in arcseconds"""
    coord1 = SkyCoord(ra1, dec1, unit=unit1)
    coord2 = SkyCoord(ra2, dec2, unit=unit2)
    return coord1.separation(coord2).arcsec

def ab_mag_to_flux(AB_mag: np.ndarray) -> np.ndarray:
    """Convert AB magnitude to flux in mJy"""
    return 10**((AB_mag - 8.9) / -2.5) * 1000

ab_magerr_to_ferr = lambda sigma_m, f: np.abs(f * np.log(10) * (sigma_m / 2.5))  # transformation on the error of a magnitude turned into flux

# Add angular separation and grib data
bts_df['angular_separation_arcsec'] = get_angular_separation(bts_df['raStack_3pi'], bts_df['decStack_3pi'], bts_df['SN_ra'], bts_df['SN_dec'], unit1='deg', unit2='deg')
grizy = bts_df[['gKronMag_3pi', 'rKronMag_3pi', 'iKronMag_3pi', 'zKronMag_3pi', 'yKronMag_3pi']].to_numpy().astype(float)
grizy_err = bts_df[['gKronMagErr_3pi', 'rKronMagErr_3pi', 'iKronMagErr_3pi', 'zKronMagErr_3pi', 'yKronMagErr_3pi']].to_numpy().astype(float)
angular_seps = bts_df['angular_separation_arcsec'].to_numpy().astype(float)

# Convert the grizy data to mJy
grizy = ab_mag_to_flux(grizy)
grizy_err = ab_magerr_to_ferr(grizy_err, grizy)

grizy, angular_seps

(array([[1.83873863e-01, 2.34509262e-01, 2.57039578e-01, 2.62252719e-01,
         2.80001269e-01],
        [1.15048245e-01, 8.64171621e-02, 1.26287394e-01, 1.06649789e-01,
         1.29169506e-01],
        [8.85115610e-01, 1.99122346e+00, 2.87051619e+00, 3.60944144e+00,
         4.53649044e+00],
        [           nan, 7.37292812e+00, 1.96607463e+01, 2.33002188e+01,
         2.74233180e+01],
        [4.48993444e-01, 5.91725111e-01, 6.50309353e-01, 6.77204757e-01,
         7.15418226e-01],
        [2.98483274e-01, 6.53792599e-01, 1.09415790e+00, 1.42902557e+00,
         1.69855651e+00],
        [6.01893981e-01, 1.23720025e+00, 1.92415476e+00, 2.25029782e+00,
         2.59680895e+00],
        [1.73988282e+00,            nan,            nan,            nan,
                    nan],
        [3.22760218e+00, 5.96815371e+00, 8.96107269e+00, 1.00110585e+01,
         1.09496440e+01],
        [4.09449175e-01, 8.94046279e-01, 1.34091114e+00, 1.71316818e+00,
         2.08276370e+00],
        [2

In [4]:
nan_mask = ~(np.sum(np.isnan(grizy), axis=1)==0)

# Use the SPLASH pipeline

In [5]:
# Load pipeline object
pipeline = Splash_Pipeline(
    pre_transformed=False,      # whether the given data is pre-logged and nnormalized
    within_4sigma=True,         # whether we only want to classify objects with properties within 4-sigma of the training set
    nan_thresh_ratio=1.0,       # to keep this notebook concise, we are allowing the pipeline to imput any num of nans
)

In [6]:
# Predict the classes. n_resamples is the number of boostraps for getting the median predicted host properties.
start_time = time.time()
classes_only_phot = pipeline.infer_classes(grizy=grizy, redshift=np.array(bts_hosts['sn_redshift'].filled(np.nan)), angular_sep=angular_seps, grizy_err=grizy_err, n_resamples=50)
duration = time.time() - start_time
print(f'{grizy.shape[0]} classifications produced in {duration} seconds (~{duration / grizy.shape[0]} seconds per classificaiton).')

99 classifications produced in 0.07047390937805176 seconds (~0.0007118576704853713 seconds per classificaiton).


In [7]:
print(f'Number of each class:\n{pd.Series(classes_only_phot).value_counts()}\n\nThe class labels are\n0=Ia\n1=Ib/c\n2=SLSN\n3=IIn\n4=II (P/L)\n-1=Outside train properties 4 sigma\n-2=No host found')

Number of each class:
0    92
4     7
Name: count, dtype: int64

The class labels are
0=Ia
1=Ib/c
2=SLSN
3=IIn
4=II (P/L)
-1=Outside train properties 4 sigma
-2=No host found


In [8]:
# We can also take a look at the predicted host properties of each supernova
props, props_err = pipeline.infer_host_properties(grizy=grizy, redshift=np.array(bts_hosts['sn_redshift'].filled(np.nan)), grizy_err=grizy_err, n_resamples=50, return_normalized=False)  # return_normalized will return the properties normalized with the train mean and std
props, props_err  # in order (log(mass), log(sfr), redshift)

TypeError: cannot unpack non-iterable NoneType object

# Classification from just transient coordinates

In [9]:
# Load pipeline object
pipeline = Splash_Pipeline(
    pre_transformed=False,      # whether the given data is pre-logged and nnormalized
    within_4sigma=True,         # whether we only want to classify objects with properties within 4-sigma of the training set
    nan_thresh_ratio=1.0,       # to keep this notebook concise, we are allowing the pipeline to imput any num of nans
)

In [10]:
classes = pipeline.infer_classes(
    ra=bts_hosts['SN_ra'][:25],
    dec=bts_hosts['SN_dec'][:25],
    redshift=np.array(bts_hosts['sn_redshift'].filled(np.nan))[:25],
    n_resamples=50,
)
print(f'Number of each class:\n{pd.Series(classes).value_counts()}\n\nThe class labels are 0=Ia 1=Ib/c 2=SLSN 3=IIn 4=II (P/L) -1=Outside train properties 4 sigma -2=No host found.')

Associating the catalog!


2025-05-22 09:02:13,449- WARNING - Association failed. Host is likely missing from the catalog.
2025-05-22 09:02:17,445- WARNING - Association failed. Host is likely missing from the catalog.
2025-05-22 09:02:30,699- WARNING - Association failed. Host is likely missing from the catalog.
2025-05-22 09:02:41,315- WARNING - Association failed. Host is likely missing from the catalog.
/Users/ashley/Programs/astro_SPLASH/SPLASH/pipeline.py:551: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  pstarr_photo_df = pd.concat([pstarr_photo_df, new_src_df[photo_cols].iloc[0:1]])


Number of each class:
 0    22
-2     2
 1     1
Name: count, dtype: int64

The class labels are 0=Ia 1=Ib/c 2=SLSN 3=IIn 4=II (P/L) -1=Outside train properties 4 sigma -2=No host found.


In [11]:
pipeline.transient_catalog[['best_cat'] + [k for k in pipeline.transient_catalog.keys() if 'host' in k]]

,best_cat,host_objID,host_name,host_total_posterior,host_ra,host_dec,host_redshift_mean,host_redshift_std,host_redshift_info,host_redshift_posterior,...,host_2_redshift_info,host_2_redshift_posterior,host_2_absmag_mean,host_2_absmag_std,host_2_absmag_info,host_2_absmag_posterior,host_2_offset_mean,host_2_offset_std,host_2_offset_info,host_2_offset_posterior
0,decals,9907732922236989,,0.999672,146.514984,33.528398,0.028631,0.010065,PHOT,547018.810156,...,PHOT,1.058666e+01,-20.575779,1.089219,r,10.665303,296.801952,0.102264,,0.000000
1,decals,9907734301903030,,0.999055,134.954724,38.108963,0.057822,0.028911,PHOT,281917.041568,...,PHOT,2.255588e+01,-20.160897,1.159131,r,7.657950,293.806932,0.096856,,0.000000
2,glade,1060433.0,2385018,0.965570,120.649139,50.922501,0.053335,0.016000,SPEC,10785.995237,...,PHOT,7.531221e+02,-19.640000,0.958500,B,0.082906,214.446633,0.095463,,0.000000
3,glade,1027259.0,UGC03554,0.993457,102.607574,43.053371,0.018141,0.005442,SPEC,6618.673557,...,SPEC,6.618674e+03,-20.732025,0.687500,B,0.051628,0.542937,0.098033,,9.861829
4,glade,2174757.0,SDSSJ083808.16+31400,0.899432,129.534000,31.667870,0.031472,0.009442,PHOT,61466.647146,...,PHOT,6.146665e+04,-18.649993,0.852500,B,0.090280,0.231688,0.089779,,86.937618
5,decals,9907735872410137,,0.998644,98.507299,43.408915,0.063597,0.005370,PHOT,19400.924857,...,PHOT,3.271253e+00,-20.796787,1.146864,r,0.174471,292.935138,0.092814,,0.000000
6,decals,9907735805166489,,0.999940,106.823710,43.316375,0.069899,0.008682,PHOT,3124.342993,...,PHOT,6.665166e-21,-21.628783,1.109849,r,1.230144,295.432118,0.088636,,0.000000
7,glade,1035402.0,21598,0.988445,115.494247,23.176296,0.043268,0.012980,SPEC,4506.622048,...,PHOT,6.808164e+01,-20.849999,0.925000,B,0.090838,188.272502,0.097184,,0.000000
8,glade,1039273.0,PGC029733,0.981075,153.178818,39.373905,0.021596,0.006479,SPEC,15303.994845,...,PHOT,2.590326e+02,-19.689998,0.903000,B,0.051714,236.446910,0.080981,,0.000000
9,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
